## Problem 1

In [3]:
def pretty_print_mtx(mtx):
    for row in mtx:
        print(''.join(row))

In [4]:
dir_vect = {
    '^': [-1, 0],
    '<': [0, -1],
    'v': [1, 0],
    '>': [0, 1]
}

def move_robot(mtx, pos, move):
    v = dir_vect[move]

    obj_to_move = 1
    curr_pos = [pos[0], pos[1]]
    while True:
        curr_pos = [curr_pos[0] + v[0], curr_pos[1] + v[1]]
        if mtx[curr_pos[0]][curr_pos[1]] == '#':
            obj_to_move = 0
            break
        if mtx[curr_pos[0]][curr_pos[1]] == '.':
            break
        obj_to_move += 1

    if obj_to_move >= 2:
        mtx[pos[0] + obj_to_move * v[0]][pos[1] + obj_to_move * v[1]] = 'O'
    if obj_to_move >= 1:
        mtx[pos[0]][pos[1]] = '.'
        mtx[pos[0] + v[0]][pos[1] + v[1]] = '@'
        pos[0] = pos[0] + v[0]
        pos[1] = pos[1] + v[1]

In [5]:
def compute_gps(mtx):
    count = 0
    for i in range(len(mtx)):
        for j in range(len(mtx[0])):
            if mtx[i][j] == 'O':
                count += 100 * i + j
    return count

In [6]:
def solve1(input_filename):
    f = open(f'../inputs/{input_filename}.txt', 'r')

    # read warehouse layout
    mtx = []
    pos = None
    row = 0
    while True:
        line = f.readline()
        if line == '\n':
            break
        mtx.append(list(line.strip()))
        if pos == None:
            col = line.find('@')
            if col != -1:
                pos = [row, col]
        row += 1

    # read & treat movements
    while True:
        line = f.readline()
        if line == '':
            break
        # treat movements
        for move in line.strip():
            move_robot(mtx, pos, move)
    f.close()

    return compute_gps(mtx)

## Problem 2

In [7]:
transform_tile = {
    '#': ['#', '#'],
    'O': ['[', ']'],
    '.': ['.', '.'],
    '@': ['@', '.']
}

def transform_mtx(mtx, pos):
    new_mtx = []
    for i in range(len(mtx)):
        new_mtx.append([])
        for j in range(len(mtx[0])):
            new_mtx[i].extend(transform_tile[mtx[i][j]])

    return (new_mtx, [pos[0], pos[1] * 2])

In [8]:
# move should be '>' or '<'
def xmove_robot(mtx, pos, move):
    v = -1 if move == '<' else 1

    tiles_to_move = 1
    curr_y = pos[1]
    while True:
        curr_y += v
        if mtx[pos[0]][curr_y] == '#':
            tiles_to_move = 0
            break
        if mtx[pos[0]][curr_y] == '.':
            break
        tiles_to_move += 1

    if tiles_to_move >= 1:
        j = pos[1] + v * tiles_to_move
        while j != pos[1]:
            mtx[pos[0]][j] = mtx[pos[0]][j - v]
            j -= v
        mtx[pos[0]][pos[1]] = '.'
        pos[1] += v

In [34]:
# move should be 'v' or '^'
def ymove_robot(mtx, pos, move):
    v = -1 if move == '^' else 1

    tiles_to_move = 1
    curr_x = pos[0]
    moving_objects = { pos[0]: { pos[1] } }
    while True:
        curr_x += v
        moving_objects[curr_x] = set()
        for y in moving_objects[curr_x - v]:
            if mtx[curr_x][y] == '#':
                tiles_to_move = 0
                break
            if mtx[curr_x][y] == '[':
                moving_objects[curr_x].add(y)
                moving_objects[curr_x].add(y+1)
            if mtx[curr_x][y] == ']':
                moving_objects[curr_x].add(y)
                moving_objects[curr_x].add(y-1)
        if tiles_to_move == 0 or len(moving_objects[curr_x]) == 0:
            break
        tiles_to_move += 1

    if tiles_to_move >= 1:
        i = pos[0] + v * tiles_to_move
        while i != pos[0]:
            for j in moving_objects[i - v]:
                mtx[i][j] = mtx[i-v][j]
                mtx[i-v][j] = '.'
            i -= v
        pos[0] += v

In [10]:
def compute_gps2(mtx):
    count = 0
    for i in range(len(mtx)):
        for j in range(len(mtx[0])):
            if mtx[i][j] == '[':
                count += 100 * i + j
    return count

In [38]:
def solve2(input_filename):
    f = open(f'../inputs/{input_filename}.txt', 'r')

    # read warehouse layout
    mtx = []
    pos = None
    row = 0
    while True:
        line = f.readline()
        if line == '\n':
            break
        mtx.append(list(line.strip()))
        if pos == None:
            col = line.find('@')
            if col != -1:
                pos = [row, col]
        row += 1
    (mtx, pos) = transform_mtx(mtx, pos)

    # read & treat movements
    while True:
        line = f.readline()
        if line == '':
            break
        # treat movements
        for move in line.strip():
            # if moves == move_num - 1:
            #     pretty_print_mtx(mtx)
            #     print(pos)
            #     print(move)
            # if moves == move_num:
            #     pretty_print_mtx(mtx)
            #     print(pos)
            #     return
            if move == '>' or move == '<':
                xmove_robot(mtx, pos, move)
            else:
                ymove_robot(mtx, pos, move)
    f.close()

    return compute_gps2(mtx)